In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

# magic function is to enable the inline plotting
%matplotlib inline 

# for display all the column in the datafarmes
pd.pandas.set_option('display.max_columns',None)

In [74]:
test = pd.read_csv("https://raw.githubusercontent.com/Nikhil-V98/Analyticsvidhya/main/Loan_prediction/test_lAUu6dG.csv")
train = pd.read_csv("https://raw.githubusercontent.com/Nikhil-V98/Analyticsvidhya/main/Loan_prediction/train_ctrUa4K.csv")

In [75]:
testRes = test[['Loan_ID']]

In [76]:
train.drop('Loan_ID',axis=1,inplace = True)
test.drop('Loan_ID',axis=1,inplace = True)

In [77]:
train['Dependents'].replace(to_replace='3+',value='3',inplace = True)
test['Dependents'].replace(to_replace='3+',value='3',inplace = True)

## Missing Value Treatment

In [78]:
cat = train[['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area','Credit_History','Loan_Amount_Term']]
train["LoanAmount"].fillna(train["LoanAmount"].median(),inplace=True)
test["LoanAmount"].fillna(test["LoanAmount"].median(),inplace=True)

for var in cat.columns:
    if train[var].isnull().sum() >0:
        train[var].fillna(train[var].mode()[0],inplace=True)

## Test data

for var in cat.columns:
    if test[var].isnull().sum() >0:
        test[var].fillna(test[var].mode()[0],inplace=True)

## Cat ro Num

In [79]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(sparse=False,drop = 'if_binary')

transformed_data = onehotencoder.fit_transform(train[['Gender']])
# the above transformed_data is an array so convert it to dataframe and add feature name to the comlum
encoded_data = pd.DataFrame(transformed_data, columns=onehotencoder.get_feature_names_out())

# now concatenate the original data and the encoded data using pandas
train = pd.concat([train, encoded_data], axis=1).drop('Gender', axis=1)

## Test data

from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(sparse=False,drop = 'if_binary')

transformed_data = onehotencoder.fit_transform(test[['Gender']])
# the above transformed_data is an array so convert it to dataframe and add feature name to the comlum
encoded_data = pd.DataFrame(transformed_data, columns=onehotencoder.get_feature_names_out())

# now concatenate the original data and the encoded data using pandas
test = pd.concat([test, encoded_data], axis=1).drop('Gender', axis=1)

In [80]:
label_en = []
for var in test.columns:
    if len(test[var].unique()) < 6 :
        label_en.append(var)
        
label_en.pop()

'Gender_Male'

In [81]:
from sklearn.preprocessing import LabelEncoder
feature_col = label_en
le = LabelEncoder()
for col in feature_col:
    train[col] = le.fit_transform(train[col])
    test[col] = le.fit_transform(test[col])

Loan_Status_dict = {'N':0, 'Y':1}
train['Loan_Status'] = train['Loan_Status'].map(Loan_Status_dict)

In [87]:
train['Loan_Amount_Term'] = le.fit_transform(train['Loan_Amount_Term'])
test['Loan_Amount_Term'] = le.fit_transform(test['Loan_Amount_Term'])

## Outlier Treatment

In [88]:
for var in ['ApplicantIncome','CoapplicantIncome','LoanAmount'] :
    q10 = train[var].quantile(0.10)
    q90 = train[var].quantile(0.90)
    train[var] = np.where(train[var] <q10, q10,train[var])
    train[var] = np.where(train[var] >q90, q90,train[var])

In [89]:
for var in ['ApplicantIncome','CoapplicantIncome','LoanAmount'] :
    q10 = test[var].quantile(0.10)
    q90 = test[var].quantile(0.90)
    test[var] = np.where(test[var] <q10, q10,test[var])
    test[var] = np.where(test[var] >q90, q90,test[var])

## Scaling

In [90]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
na_features = ['ApplicantIncome','CoapplicantIncome','LoanAmount']
for var in na_features:
    train[var] = scaler.fit_transform(train[var].values.reshape(-1, 1))
    test[var] = scaler.transform(test[var].values.reshape(-1, 1))

In [91]:
from sklearn.model_selection import train_test_split
X = train.drop(['Loan_Status'], axis=1)
y = train[['Loan_Status']]
X_test = test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)

In [92]:
from collections import OrderedDict
model_performance = OrderedDict()

In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logistic_model = LogisticRegression(random_state=1)
lg_model = logistic_model.fit(X_train,y_train)

y_pred_logistic=lg_model.predict(X_val)

score_logistic =accuracy_score(y_pred_logistic,y_val)*100

model_performance['Multi Linear Regression'] = round(score_logistic,3)
print(f'Root Mean Squared Error of the model is : {score_logistic}')

Root Mean Squared Error of the model is : 78.86178861788618


c:\Users\Nik\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [94]:
from sklearn.tree import DecisionTreeClassifier
tree_model = DecisionTreeClassifier(random_state=1)
tree_model.fit(X_train,y_train)
pred_cv_tree=tree_model.predict(X_val)
score_tree =accuracy_score(pred_cv_tree,y_val)*100 

model_performance['Decision Tree'] = round(score_tree,3)
print(f'Root Mean Squared Error of the model is : {score_tree}')

Root Mean Squared Error of the model is : 74.79674796747967


In [95]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier(random_state=1,max_depth=10,n_estimators=50)
forest_model.fit(X_train,y_train)
pred_cv_forest=forest_model.predict(X_val)
score_forest = accuracy_score(pred_cv_forest,y_val)*100
model_performance['Random Forest_10'] = round(score_forest,3)
print(f'Root Mean Squared Error of the model is : {score_forest}')

Root Mean Squared Error of the model is : 78.04878048780488


C:\Users\Nik\AppData\Local\Temp\ipykernel_5044\722122929.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_model.fit(X_train,y_train)


In [96]:
from sklearn.model_selection import GridSearchCV
paramgrid = {'max_depth': list(range(1,20,2)),'n_estimators':list(range(1,200,20))}
grid_search = GridSearchCV(RandomForestClassifier(random_state=1),paramgrid)
grid_search.fit(X_train,y_train)
grid_search.best_estimator_

c:\Users\Nik\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\Nik\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\Nik\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\Nik\AppData\Local\Programs\Python\Pyt

RandomForestClassifier(max_depth=3, n_estimators=81, random_state=1)

In [97]:
grid_forest_model = RandomForestClassifier(random_state=1,max_depth=3,n_estimators=81)
grid_forest_model.fit(X_train,y_train)
pred_grid_forest = grid_forest_model.predict(X_val)
score_grid_forest = accuracy_score(pred_grid_forest,y_val)*100
model_performance['Random Forest with Grid-search'] = round(score_grid_forest,3)
print(f'Root Mean Squared Error of the model is : {score_grid_forest}')

Root Mean Squared Error of the model is : 78.86178861788618


C:\Users\Nik\AppData\Local\Temp\ipykernel_5044\108415849.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  grid_forest_model.fit(X_train,y_train)


In [98]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(n_estimators=50,max_depth=4) 
xgb_model.fit(X_train,y_train)
pred_xgb=xgb_model.predict(X_val)
score_xgb = accuracy_score(pred_xgb,y_val)*100
model_performance['XGBoost'] = round(score_xgb,3)
print(f'Root Mean Squared Error of the model is : {round(score_xgb,3)}')

Root Mean Squared Error of the model is : 78.862


In [99]:
model_performance

OrderedDict([('Multi Linear Regression', 78.862),
             ('Decision Tree', 74.797),
             ('Random Forest_10', 78.049),
             ('Random Forest with Grid-search', 78.862),
             ('XGBoost', 78.862)])

In [101]:
yPreds = grid_forest_model.predict(X_test)
testRes['Loan_Status'] = yPreds


In [102]:
Loan_Status_dict_1= {0:'N', 1:'Y'}
testRes['Loan_Status'] = testRes['Loan_Status'].map(Loan_Status_dict_1)

In [107]:
submission = testRes[['Loan_ID', 'Loan_Status']]
submission.columns = ['Loan_ID','Loan_Status']
submission.to_csv('submission_v3.csv', index = False)
submission.head()

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,Y
